# Fine-tune Stable Diffusion using Dreambooth with LoRA

In this notebook, we show how to fine-tune a Stable Diffusion model using your own dataset but specifically utilizing the low rank adaptation technique, LoRA, which allows us to fine-tune the cross-attention layers without performing full fine-tuning. 

To do this, we'll utilize some Hugging Face libraries including the [Diffuser's Library](https://huggingface.co/docs/diffusers/index) and [PEFT Libraries](https://huggingface.co/docs/peft/index). 

Once again, we'll focus on fine-tuning using a few images of Molly dog as a puppy! 

# Setup

First, we'll perform some setup by installing dependencies.  Please restart the kernel after all dependencies have been installed.

In [2]:
!pip install transformers accelerate>=0.20.3 ftfy tensorboard Jinja2 huggingface_hub wandb kaggle git+https://github.com/huggingface/diffusers

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-ruhwabsa

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
! pip install -U accelerate
! pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/13/9e/ee987874058f2d93006961f6ff49e0bcb60ab9c26709ebe06bfa8707a4d8/accelerate-0.24.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 114.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 47.2 MB/s eta 

In [4]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.35.2', '0.24.1')

In [5]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py

--2023-11-17 20:01:59--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59226 (58K) [text/plain]
Saving to: ‘train_dreambooth_lora.py.1’

train_dreambooth_lo 100%[===================>]  57.84K  --.-KB/s    in 0.001s  

2023-11-17 20:01:59 (100 MB/s) - ‘train_dreambooth_lora.py.1’ saved [59226/59226]



In [6]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/requirements.txt

--2023-11-17 20:01:59--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      76  --.-KB/s    in 0s      

2023-11-17 20:02:00 (2.58 MB/s) - ‘requirements.txt’ saved [76/76]



In [7]:
!pip install -r requirements.txt
!pip install git+https://github.com/huggingface/peft

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-esl10m7y
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-esl10m7y
  Resolved https://github.com/huggingface/peft to commit f1ecfa6ae6eba599ae89decbf47b339d8c9d39a3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
!pwd

/root/generative-ai-on-aws/11_controlnet


In [9]:
#!export INSTANCE_DIR='/home/sagemaker-user/dreambooth_lora/training_images'
#!export CLASS_DIR='/home/sagemaker-user/dreambooth_lora/base_class'

In [10]:
import shutil, os
dog_images = os.listdir("images-lora-molly")

In [11]:
!mkdir -p images-lora-molly-p
!mkdir -p images-lora-base-class

In [12]:
for filename in dog_images:
    shutil.copyfile(
         os.path.join("images-lora-molly-p",filename),
         os.path.join("images-lora-base-class",filename)
   )

FileNotFoundError: [Errno 2] No such file or directory: 'images-lora-molly-p/Molly_p_2.jpg'

In [ ]:
%autosave 0

In [13]:
instance_data_directory = '/home/sagemaker-user/generative-ai-on-aws/11_controlnet/images-lora-molly-p'

Next, we'll use a training script adapted from a Hugging Face example to fine-tune a Stable Diffusion model, specifically verson 1.5, using LoRA fine-tuning.  Also, to show variety in how you can choose to uilitize SageMaker as well as common libraries, we are showing training being locally performed on this SageMaker Studio notebook instance versus separate compute through spinning up training jobs.  This can be a good option for experimentation and you can easily [change the size of your SageMaker notebook](https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-run-and-manage-switch-instance-type.html) on the fly to support this type of experimentation.

We'll be using another Hugging Face library called [Accelerate](https://huggingface.co/docs/accelerate/index) which provides a variety of helpful libraries for running training and inference.

Note, in this case we are specifying the instance and class prompt directly in the configuration supplied instead of separately in the dataset_info.json as previously shown in the full fine-tuning notebook (03_dreambooth.ipynb).   

A few parameters to point out below include: 

  * **lora_r**: This parameter indicates the rank of the low rank matrices where a lower rank results in smaller matrices with fewer trainable parameters.
  * **lora_alpha**: This parameter notes the scaling factor to be used and controls how much importance you want to give to the new updated weight matrices when combining it with the original pretrained weights. 
  * **lr_scheduler**: This parameter controls how the learning rate changes during training. In the example below, we set it to 'constant' meaning that the learning rate will remain the same during training. 
  * **lr_warmup_steps**: This parameter specifies the number of warmup steps.

In [14]:
#https://huggingface.co/docs/peft/task_guides/dreambooth_lora#finetuning-dreambooth
!accelerate launch train_dreambooth_hf_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5"  \
  --instance_data_dir="images-lora-molly-p" \
  --class_data_dir="images-lora-base-class" \
  --output_dir=trained_model \
  --train_text_encoder \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of molly dog" \
  --class_prompt="a photo of a sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --use_lora \
  --lora_r 16 \
  --lora_alpha 27 \
  --lora_text_encoder_r 16 \
  --lora_text_encoder_alpha 17 \
  --learning_rate=1e-4 \
  --gradient_accumulation_steps=1 \
  --gradient_checkpointing \
  --max_train_steps=800

/opt/conda/bin/python: can't open file '/root/generative-ai-on-aws/11_controlnet/train_dreambooth_hf_lora.py': [Errno 2] No such file or directory
/opt/conda/bin/python: can't open file '/root/generative-ai-on-aws/11_controlnet/train_dreambooth_hf_lora.py': [Errno 2] No such file or directory
/opt/conda/bin/python: can't open file '/root/generative-ai-on-aws/11_controlnet/train_dreambooth_hf_lora.py': [Errno 2] No such file or directory
/opt/conda/bin/python: can't open file '/root/generative-ai-on-aws/11_controlnet/train_dreambooth_hf_lora.py': [Errno 2] No such file or directory
/opt/conda/bin/python: can't open file '/root/generative-ai-on-aws/11_controlnet/train_dreambooth_hf_lora.py': [Errno 2] No such file or directory
/opt/conda/bin/python: can't open file '/root/generative-ai-on-aws/11_controlnet/train_dreambooth_hf_lora.py': [Errno 2] No such file or directory
/opt/conda/bin/python: can't open file '/root/generative-ai-on-aws/11_controlnet/train_dreambooth_hf_lora.py': [Errno 

## Run inference against fine-tuned weights

Next, let's run inference against the fine-tune weights using the base model and the new fine-tuned LoRA weights.

In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

In [ ]:
pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

In [ ]:
def get_lora_sd_pipeline(
    ckpt_dir, base_model_name_or_path=None, dtype=torch.float16, device="cuda", adapter_name="molly"
):
    unet_sub_dir = os.path.join(ckpt_dir, "unet")
    text_encoder_sub_dir = os.path.join(ckpt_dir, "text_encoder")
    if os.path.exists(text_encoder_sub_dir) and base_model_name_or_path is None:
        config = LoraConfig.from_pretrained(text_encoder_sub_dir)
        base_model_name_or_path = config.base_model_name_or_path

    if base_model_name_or_path is None:
        raise ValueError("Please specify the base model name or path")

    pipe = StableDiffusionPipeline.from_pretrained(base_model_name_or_path, torch_dtype=dtype).to(device)
    pipe.unet = PeftModel.from_pretrained(pipe.unet, unet_sub_dir, adapter_name=adapter_name)

    if os.path.exists(text_encoder_sub_dir):
        pipe.text_encoder = PeftModel.from_pretrained(
            pipe.text_encoder, text_encoder_sub_dir, adapter_name=adapter_name
        )

    if dtype in (torch.float16, torch.bfloat16):
        pipe.unet.half()
        pipe.text_encoder.half()

    pipe.to(device)
    return pipe

In [ ]:
from pathlib import Path
from peft import PeftModel, LoraConfig
from diffusers import StableDiffusionPipeline

pipe = get_lora_sd_pipeline(Path("./trained_model"), adapter_name="molly")

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

Now, let's prompt the model (in this case the base foundation model combined with the LoRA adapter weights) for a picture of Molly on the beach. 

In [ ]:
img_list = pipe(["Molly dog on a beach"]*3, num_inference_steps=50).images
image_grid([x.resize((128,128)) for x in img_list], 1,3)

# Congratulations! 

You fine-tuned Stable Diffusion models using a parameter efficient, light weight method that still allows for customization of the model based on your own dataset.  It's also suggested to review the size of the fine-tuned weights.  You'll notice the the fine-tuned weights are a fraction of the size of the full model allowing you to easily customize the model across a number of custom classes or subjects.  

In this case, we used our photos of Molly dog to fine-tune but feel free to try out your own examples.